## Create text-based backups of analyses and transformations (for git)


### This script backs up all analyses and transformations into a backup folder, the user only needs to provide a Celonis object and a folder.


### Do imports, log in to the Celonis instance, create backup folder

In [1]:
from pycelonis import get_celonis
import shutil
from pathlib import Path
from pycelonis.utils.api_utils import pathify

celonis = get_celonis(read_only=True)
backup_path = Path('IBC Backup')
if not backup_path.exists():
    backup_path.mkdir()

2019-10-08 15:36:00 - Login successful! Hello Simon Riezebos


### Create or clean analyses backup folder

In [3]:
analyses_path = backup_path / "Analyses"
if analyses_path.exists():
    shutil.rmtree(analyses_path)
analyses_path.mkdir()

### Create backups of all analyses that are published in separate workspace directories

In [4]:
for workspace in celonis.workspaces:
    workspace_path = analyses_path / pathify(workspace.name)
    workspace_path.mkdir()
    for a in workspace.analyses:
        if a.data.get('lastPublishedDraftId') is not None:
            a.backup_content(workspace_path)

### (Optional) Remove all draft files to only see published changes

In [5]:
for path in analyses_path.rglob('*'):
    if path.name.startswith("draft") and path.suffix == ".json":
        path.unlink()

### Create or clean transformation backup folder

In [4]:
transformation_path = backup_path / "Transformations"
if transformation_path.exists():
    shutil.rmtree(transformation_path)
transformation_path.mkdir()

### Create backups of all transformations in separate Pool and Data Job directories

In [5]:
for pool in celonis.pools:
    pool_path = transformation_path / pathify(pool.name)
    pool_path.mkdir()
    for job in pool.data_jobs:
        job_path = pool_path / pathify(job.name)
        job_path.mkdir()
        for tm in job.transformations:
            try:
                tm.backup_content(job_path)
            except:
                pass

### (Optional) Inititate git repo
Navigate to the back_path on the command line, run:
```
git init
git add .
git commit -m "Activating version control"
```
When you re-run this notebook all changes will be easy to inspect, and can be committed again